In [1]:
import numpy as np
import pytest
import pdb

np.random.seed(42)

In [2]:
# compute CE cash flow with a mortality curve

# compute % of original cohort that dies each year
# mortality starts at 1% in year one and goes up linearly by 1% each year
mortality = np.linspace(0.00, 1.0, 101)
survivorship = np.cumprod(1-mortality[:31])
deathrate = -np.diff(survivorship)
# top up last 1 to get to 
deathrate[-1] += 1- np.sum(deathrate)
print ("sum", np.sum(deathrate))
deathrate



sum 1.0


array([0.01      , 0.0198    , 0.029106  , 0.03764376, 0.04517251,
       0.05149666, 0.05647467, 0.06002451, 0.06212537, 0.06281565,
       0.06218749, 0.0603784 , 0.05756074, 0.05392999, 0.04969263,
       0.04505465, 0.04021128, 0.03533862, 0.03058754, 0.0260799 ,
       0.02190712, 0.01813075, 0.0147848 , 0.01187927, 0.00940442,
       0.00733545, 0.00563701, 0.00426742, 0.00318228, 0.0077911 ])

In [3]:
# random cashflows
cashflows = np.random.uniform(1, 100, 30)
cashflows

array([38.07947177, 95.12071633, 73.46740024, 60.26718994, 16.4458454 ,
       16.44345751,  6.7502776 , 86.75143843, 60.51038616, 71.0991852 ,
        3.03786494, 97.02107536, 83.41182144, 22.02157196, 19.00067175,
       19.15704648, 31.11998205, 52.95088673, 43.76255685, 29.83168488,
       61.57343658, 14.8098922 , 29.9223202 , 37.26982249, 46.15092844,
       78.73242018, 20.76770443, 51.9092094 , 59.64904232,  5.59859086])

In [4]:
# utility, for example log utility (gamma = 1)
u = np.log(cashflows)
u

array([3.63967534, 4.55514678, 4.29684177, 4.09878784, 2.80007289,
       2.79992768, 1.90958363, 4.463047  , 4.10281502, 4.26407588,
       1.11115494, 4.57492823, 4.42379004, 3.09202252, 2.94447433,
       2.95267061, 3.43785012, 3.96936482, 3.77877859, 3.39557108,
       4.12023055, 2.69529535, 3.3986047 , 3.61818395, 3.83191708,
       4.36605502, 3.03339911, 3.94949622, 4.08847809, 1.72251493])

In [5]:
indices = np.indices(u.shape)[0]+1
indices

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

In [6]:
# cumulative mean utility
mean_u = np.cumsum(u)/indices
mean_u

array([3.63967534, 4.09741106, 4.16388797, 4.14761293, 3.87810492,
       3.69840872, 3.44286228, 3.57038537, 3.62954422, 3.69299738,
       3.45828443, 3.55133808, 3.61844977, 3.58084783, 3.53842293,
       3.50181341, 3.49805086, 3.52423497, 3.537632  , 3.53052896,
       3.55860998, 3.51936841, 3.51411781, 3.5184539 , 3.53099243,
       3.56311022, 3.54349129, 3.55799147, 3.57628411, 3.5144918 ])

In [7]:
# convert utility back to cash flow and multiply by number of years for cumulative ce cash flow
ce = np.exp(mean_u) * indices
ce

array([  38.07947177,  120.3685446 ,  192.96334598,  253.13103899,
        241.66267216,  242.2979546 ,  218.93446207,  284.24226157,
        339.26068512,  401.65056775,  349.38683112,  418.31918156,
        484.63650235,  502.65555566,  516.18907746,  530.80893156,
        561.86645922,  610.70054169,  653.32256476,  682.84044909,
        737.40142762,  742.78814031,  772.48456618,  809.57363536,
        853.94625157,  917.09105093,  933.86151596,  982.59396472,
       1036.47409588, 1007.96546332])

In [8]:
# experienced cumulative CE cash flow 
# multiply each year's % who died in that year by cumulative CE cash flow up to that year
# and then sum for average experienced total cash flow
madj_ce = np.sum(ce * deathrate)
madj_ce

# (there's a (weak) argument for using the average, 
# i.e. adjusting to equal weight the guy who died in year 1 with the guy who died in year 30)


436.58452788516126

In [9]:
def crra_ce(cashflows, gamma):
    """takes a numpy array, returns total CRRA certainty-equivalent cash flow"""
    # for retirement study assume no negative cashflows
    if sum(np.where(cashflows < 0, 1, 0)):
        return 0.0
    elif gamma >= 1.0 and 0 in cashflows:
        return 0.0
    elif gamma == 1.0:
        # general formula for CRRA utility undefined for gamma = 1 but limit as gamma->1 = log
        u = np.mean(np.log(cashflows))
        ce = np.exp(u)
    elif gamma == 2.0:  # simple optimization
        u2 = np.mean(1 - 1.0 / cashflows)
        ce = 1.0 / (1.0 - u2)
    elif gamma > 4.0:
        # force computations as longdouble for more precision, but always return np.float
        gamma = np.longdouble(gamma)
        cashflows = cashflows.astype(np.longdouble)
        gamma_m1 = gamma - 1.0
        gamma_m1_inverse = 1.0 / gamma_m1
        u = np.mean(gamma_m1_inverse - 1.0 / (gamma_m1 * cashflows ** gamma_m1))
        ce = 1.0 / (1.0 - gamma_m1 * u) ** gamma_m1_inverse
        ce = np.float(ce.astype)
    elif gamma > 1.0:
        gamma_m1 = gamma - 1.0
        gamma_m1_inverse = 1.0 / gamma_m1
        u = np.mean(gamma_m1_inverse - 1.0 / (gamma_m1 * cashflows ** gamma_m1))
        ce = 1.0 / (1.0 - gamma_m1 * u) ** gamma_m1_inverse
    else:  # general formula
        g_1m = 1 - gamma
        u = np.mean((cashflows ** g_1m - 1.0) / g_1m)
        ce = (g_1m * u + 1.0) ** (1.0 / g_1m)

    return ce * len(cashflows)

In [10]:
def crra_ce_deathrate(cashflows, gamma, deathrate):
    """ce cash flow with a mortality curve
    cash flows = real cash flows in each year of cohort
    gamma = risk aversion
    death rate = % of cohort that died in each year of cohort
    
    compute utility of each cash flow under gamma
    compute cumulative mean of utilities up to each year
    convert utilities back to CE cash flows
    each member of cohort that died in a given year experienced CE cash flow * years alive
    """
    # for retirement study assume no negative cashflows
    if sum(np.where(cashflows < 0, 1, 0)):
        return 0.0
    else:
        # 1..lastyear
        indices = np.indices(cashflows.shape)[0]+1
        
        if gamma == 1.0:
            # utility
            u = np.log(cashflows)
            # cumulative mean utility
            u_mean = np.cumsum(u)/indices
            # cumulative mean ce cash flows
            ce = np.exp(u_mean) * indices
        elif gamma == 2.0:  # simple optimization
            u2 = 1 - 1.0 / cashflows
            u2_mean = np.cumsum(u)/indices
            ce = 1.0 / (1.0 - u2_mean) * indices
        elif gamma > 4.0:
            # force computations as longdouble for more precision, but always return np.float
            gamma = np.longdouble(gamma)
            cashflows = cashflows.astype(np.longdouble)
            gamma_m1 = gamma - 1.0
            gamma_m1_inverse = 1.0 / gamma_m1
            u = gamma_m1_inverse - 1.0 / (gamma_m1 * cashflows ** gamma_m1)
            u_mean = np.cumsum(u)/indices
            ce = 1.0 / (1.0 - gamma_m1 * u_mean) ** gamma_m1_inverse
            ce = (ce * indices).astype(float)
        elif gamma > 1.0:
            gamma_m1 = gamma - 1.0
            gamma_m1_inverse = 1.0 / gamma_m1
            u = gamma_m1_inverse - 1.0 / (gamma_m1 * cashflows ** gamma_m1)
            u_mean = np.cumsum(u)/indices
            ce = 1.0 / (1.0 - gamma_m1 * u_mean) ** gamma_m1_inverse
            ce = ce * indices
        else:  # general formula
            gamma_1m = 1 - gamma
            u = (cashflows ** gamma_1m - 1.0) / gamma_1m
            u_mean = np.cumsum(u)/indices
            ce = (gamma_1m * u_mean + 1.0) ** (1.0 / gamma_1m)
            ce = ce * indices
        # mortality_adjusted ce cash flows
        madj_ce = np.sum(ce * deathrate)
        return madj_ce


In [11]:
testseries = np.random.uniform(1, 100, 100)
testseries.astype(np.float128)


array([61.14694034, 17.88188825,  7.44010771, 94.93966819, 96.59757127,
       81.03133746, 31.15676315, 10.66953929, 68.73906962, 44.57509688,
       13.08178525, 50.0225141 ,  4.40446359, 91.02271981, 26.61921818,
       66.58970615, 31.85939653, 52.4867341 , 55.12431765, 19.3005911 ,
       96.98887815, 77.73814951, 94.01039521, 89.58790769, 60.1920979 ,
       92.26554927,  9.7607577 , 20.40230338,  5.4775016 , 33.20770275,
       39.47905168, 27.86355415, 83.04501341, 36.31857934, 28.81251646,
       54.72691223, 14.95149827, 80.41750109,  8.38051372, 98.70180672,
       77.45223216, 20.67285247,  1.5466896 , 81.73068142, 70.97887704,
       73.17170964, 77.35576432,  8.33042052, 36.48810713, 12.47103689,
       86.44723916, 62.70651456, 33.75890446,  7.29227668, 31.78724985,
       33.19314888, 73.23101166, 64.11818966, 88.83406152, 47.74927759,
       12.83983035, 71.61123394, 76.31771981, 56.56644256, 77.32575082,
       49.88576404, 52.75055011, 43.32656082,  3.51649355, 11.68

In [12]:
# test 0 and -1
test2 = testseries.copy()
test2[50] = 0.0
assert crra_ce(test2, 1) == 0, "bad value"
test2[50] = -1.0
assert crra_ce(test2, 1) == 0, "bad value"


In [13]:
# test a short series with gammas 0, 0.5, 1, 2, 4, 8 up to 8
# test random series
# supply same random series with deathrate all 0s up until final year